In [1]:
%matplotlib inline
import json
import pandas as pd
import numpy as np

from keras import backend as K
import keras
from sklearn.metrics import log_loss
from keras.layers.convolutional import Convolution2D,ZeroPadding2D,MaxPooling2D,Conv2D,Conv1D
from keras.layers.core import Dense,Lambda,Dropout,Flatten
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD,RMSprop,Adam
from keras.models import Sequential,Model
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.regularizers import l2,l1
K.set_image_dim_ordering('th')

Using Theano backend.


In [2]:
train_data = json.load(open('train.json'))

In [3]:
train_data[0].keys()

[u'is_iceberg', u'inc_angle', u'band_2', u'id', u'band_1']

In [4]:
images_band_1 = []
images_band_2 = []
for d in train_data:
    images_band_1.append(np.array(d['band_1']))
    images_band_2.append(np.array(d['band_2']))
images_band_1 = np.array(images_band_1)
images_band_2 = np.array(images_band_2)
labels = np.array([row['is_iceberg'] for row in train_data])

print np.bincount(labels)
#labels = to_categorical(labels)
del train_data
print images_band_1.shape,images_band_2.shape,labels.shape
print images_band_1.dtype,images_band_2.dtype,labels.dtype
print labels[:5]

[851 753]
(1604, 5625) (1604, 5625) (1604,)
float64 float64 int64
[0 0 1 0 0]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler

X_train_b1,X_test_b1,y_train_b1,y_test_b1 = train_test_split(images_band_1,labels,
                                                       train_size=0.80,
                                                       stratify=labels)
X_train_b2,X_test_b2,y_train_b2,y_test_b2 = train_test_split(images_band_1,labels,
                                                       train_size=0.80,
                                                       stratify=labels)
sc_b1 = MinMaxScaler()
sc_b1.fit(X_train_b1)
X_train_b1 = sc_b1.transform(X_train_b1)
X_test_b1 = sc_b1.transform(X_test_b1)

sc_b2 = MinMaxScaler()
sc_b2.fit(X_train_b1)
X_train_b2 = sc_b2.transform(X_train_b2)
X_test_b2 = sc_b2.transform(X_test_b2)

print (X_train_b1.shape,X_train_b2.shape)
print(X_test_b1.shape,X_test_b2.shape)
print (y_train_b1.shape,y_train_b2.shape)
print (y_test_b1.shape,y_test_b2.shape)
del images_band_1,images_band_2

((1283, 5625), (1283, 5625))
((321, 5625), (321, 5625))
((1283,), (1283,))
((321,), (321,))


In [6]:
X_train_b1 = np.array([np.array(img.reshape(75,75)) for img in X_train_b1])
X_test_b1 = np.array([np.array(img.reshape(75,75)) for img in X_test_b1])

X_train_b2 = np.array([np.array(img.reshape(75,75)) for img in X_train_b2])
X_test_b2 = np.array([np.array(img.reshape(75,75)) for img in X_test_b2])

X_train_b1 = X_train_b1[:,np.newaxis,:,:]
X_test_b1 = X_test_b1[:,np.newaxis,:,:]
X_train_b2 = X_train_b2[:,np.newaxis,:,:]
X_test_b2 = X_test_b2[:,np.newaxis,:,:]

print X_train_b1.shape,X_test_b1.shape,X_train_b2.shape,X_test_b2.shape

(1283, 1, 75, 75) (321, 1, 75, 75) (1283, 1, 75, 75) (321, 1, 75, 75)


In [7]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True)
batches_b1 = gen.flow(X_train_b1,y_train_b1,batch_size=1500)
val_batches_b1 = gen.flow(X_test_b1,y_test_b1,batch_size=500)
batches_b2 = gen.flow(X_train_b2,y_train_b2,batch_size=1500)
val_batches_b2 = gen.flow(X_test_b2,y_test_b2,batch_size=500)

del X_train_b1,y_train_b1,X_test_b1,y_test_b1,X_train_b2,y_train_b2,X_test_b2,y_test_b2

model_b1 = Sequential()
model_b1.add(Lambda(lambda x:x,input_shape=(1,75,75),output_shape=(1,75,75)))
model_b1.add(BatchNormalization())
#model_b1.add(ZeroPadding2D(((1,2),(1,2))))
#model_b1.add(Conv2D(16,(3,3),activation='relu'))
#model_b1.add(MaxPooling2D((2,2)))
#model_b1.add(ZeroPadding2D(((1,0),(1,0))))
model_b1.add(Conv2D(32,(3,3),activation='relu'))
model_b1.add(MaxPooling2D((2,2)))
#model_b1.add(ZeroPadding2D(((1,0),(1,0))))
#model_b1.add(Conv2D(64,(3,3),activation='relu'))
#model_b1.add(MaxPooling2D((2,2)))
model_b1.add(Flatten())
model_b1.add(Dense(1024,activation='relu',activity_regularizer=l2(0.001)))
model_b1.add(BatchNormalization())
model_b1.add(Dropout(0.60))
model_b1.add(Dense(1024,activation='relu',activity_regularizer=l2(0.001)))
model_b1.add(BatchNormalization())
model_b1.add(Dropout(0.60))
model_b1.add(Dense(1,activation='sigmoid'))
model_b1.compile(optimizer=RMSprop(lr=0.01),loss='binary_crossentropy',metrics=['accuracy'])

model_b1.summary()

#for layer in model.layers: print layer.trainable
model_b1.fit(X_train_b1,y_train_b1,validation_data=(X_test_b1,y_test_b1),epochs=5,batch_size=16)
model_b1.optimizer.lr = 0.001
model_b1.fit(X_train_b1,y_train_b1,validation_data=(X_test_b1,y_test_b1),epochs=5,batch_size=16)
#model_b1.optimizer.lr = 0.0001
#model_b1.fit(X_train_b1,y_train_b1,validation_data=(X_test_b1,y_test_b1),epochs=3,batch_size=16)
#model_b1.optimizer.lr = 0.00001
#model_b1.fit(X_train_b1,y_train_b1,validation_data=(X_test_b1,y_test_b1),epochs=2,batch_size=16)
#model_b1.optimizer.lr = 0.000001
#model_b1.fit(X_train_b1,y_train_b1,validation_data=(X_test_b1,y_test_b1),epochs=3,batch_size=16)

In [8]:
model_b2 = Sequential()
model_b2.add(Lambda(lambda x:x,input_shape=(1,75,75),output_shape=(1,75,75)))
model_b2.add(BatchNormalization())
model_b2.add(ZeroPadding2D(((1,0),(1,0))))
model_b2.add(Conv2D(64,(3,3),activation='relu'))
model_b2.add(MaxPooling2D((2,2)))
model_b2.add(ZeroPadding2D(((1,0),(1,0))))
model_b2.add(Conv2D(128,(3,3),activation='relu'))
model_b2.add(MaxPooling2D((2,2)))
model_b2.add(ZeroPadding2D(((1,0),(1,0))))
model_b2.add(Conv2D(256,(3,3),activation='relu'))
model_b2.add(MaxPooling2D((2,2)))
model_b2.add(ZeroPadding2D(((1,0),(1,0))))
model_b2.add(Conv2D(512,(3,3),activation='relu'))
model_b2.add(MaxPooling2D((2,2)))
model_b2.add(Flatten())
model_b2.add(Dense(2048,activation='relu'))
model_b2.add(BatchNormalization())
model_b2.add(Dropout(0.75))
model_b2.add(Dense(1024,activation='relu'))
model_b2.add(BatchNormalization())
model_b2.add(Dropout(0.75))
#model_b2.add(Dense(512,activation='relu'))
#model_b2.add(BatchNormalization())
#model_b2.add(Dropout(0.75))
model_b2.add(Dense(1,activation='sigmoid'))
model_b2.compile(optimizer=RMSprop(lr=0.01),loss='binary_crossentropy',metrics=['accuracy'])

#model_b2.summary()

In [9]:
#for layer in model.layers: print layer.trainable
for i in range(10):
    batch = next(batches_b2)
    val_batch = next(val_batches_b2)
    model_b2.fit(batch[0],batch[1],validation_data=(val_batch[0],val_batch[1]),epochs=3,batch_size=16)

    model_b2.optimizer.lr = 0.001
    model_b2.fit(batch[0],batch[1],validation_data=(val_batch[0],val_batch[1]),epochs=3,batch_size=16)

    model_b2.optimizer.lr = 0.0001
    model_b2.fit(batch[0],batch[1],validation_data=(val_batch[0],val_batch[1]),epochs=3,batch_size=16)
#model_b2.optimizer.lr = 0.0001
#model_b2.fit(X_train_b2,y_train_b2,validation_data=(X_test_b2,y_test_b2),epochs=3,batch_size=16)
    model_b2.optimizer.lr = 0.00001
    model_b2.fit(batch[0],batch[1],validation_data=(val_batch[0],val_batch[1]),epochs=3,batch_size=16)

    model_b2.optimizer.lr = 0.000001
    model_b2.fit(batch[0],batch[1],validation_data=(val_batch[0],val_batch[1]),epochs=3,batch_size=16)

Train on 1283 samples, validate on 321 samples
Epoch 1/3
1283/1283 [==============================] - 246s - loss: 1.3640 - acc: 0.5066 - val_loss: 6.9060 - val_acc: 0.5047
Epoch 2/3
1283/1283 [==============================] - 220s - loss: 1.0371 - acc: 0.5347 - val_loss: 4.4243 - val_acc: 0.5202
Epoch 3/3
1283/1283 [==============================] - 224s - loss: 0.9384 - acc: 0.5012 - val_loss: 2.5043 - val_acc: 0.5826
Train on 1283 samples, validate on 321 samples
Epoch 1/3
1283/1283 [==============================] - 220s - loss: 0.8697 - acc: 0.5261 - val_loss: 0.9987 - val_acc: 0.4829
Epoch 2/3
1283/1283 [==============================] - 219s - loss: 0.7757 - acc: 0.5682 - val_loss: 1.9309 - val_acc: 0.4174
Epoch 3/3
1283/1283 [==============================] - 230s - loss: 0.7343 - acc: 0.5892 - val_loss: 0.8297 - val_acc: 0.4704
Train on 1283 samples, validate on 321 samples
Epoch 1/3
1283/1283 [==============================] - 228s - loss: 0.7356 - acc: 0.5947 - val_loss: 0.

KeyboardInterrupt: 

fig = mplot.figure(figsize=(12,12))
for i in range(16):
    fig.add_subplot(4,4,(i+1))
    mplot.text(0,0,y_train_b1[i])
    mplot.imshow(X_train_b1[i][0])

fig = mplot.figure(figsize=(12,12))
for i in range(16):
    fig.add_subplot(4,4,(i+1))
    mplot.text(0,0,y_train_b2[i+16])
    mplot.imshow(X_train_b2[i+16][0])